In [1]:
import geopandas as gpd
import os
import pandas as pd
import shutil
import rasterio
from rasterio.mask import mask
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from PIL import Image
from shapely.geometry import mapping



In [2]:
# Muat shapefile
shapefile_path = "Batas_Aceh_Utara/final_desa_1111_2022_1.shp"
desa_gdf = gpd.read_file(shapefile_path)

desa_gdf.head()


,gid,luas,kdkab,kdkec,nmkab,nmkec,iddesa,kddesa,kdprov,nmdesa,nmprov,rw_dki,sumber,periode,geometry
0,42308.0,0.242021,11,050,ACEH UTARA,MEURAH MULIA,1111050025,025,11,MEUNASAH NGA,ACEH,None,BPS,2022_1,"POLYGON ((97.21119 5.07067, 97.21121 5.07067, ..."
1,42309.0,0.590194,11,050,ACEH UTARA,MEURAH MULIA,1111050026,026,11,UJONG REUBA,ACEH,None,BPS,2022_1,"POLYGON ((97.22765 5.07125, 97.22738 5.06984, ..."
2,42310.0,0.424203,11,130,ACEH UTARA,SAMUDERA,1111130020,020,11,MURONG,ACEH,None,BPS,2022_1,"POLYGON ((97.20751 5.12251, 97.20727 5.12265, ..."
3,42311.0,0.596308,11,050,ACEH UTARA,MEURAH MULIA,1111050027,027,11,MNS MESJID,ACEH,None,BPS,2022_1,"POLYGON ((97.21815 5.07154, 97.21814 5.07145, ..."
4,42312.0,0.506486,11,020,ACEH UTARA,NISAM,1111020011,011,11,BLANG DALAM TUNONG,ACEH,None,BPS,2022_1,"POLYGON ((97.03749 5.15352, 97.03749 5.15353, ..."


### Klip Citra Daylight Sesuai dengan Batas Desa (SHP)

In [15]:
def normalize_and_convert(image):
    # Normalisasi data untuk setiap saluran
    image = np.array([(band - band.min()) / (band.max() - band.min()) * 255 for band in image])
    # Konversi ke uint8
    image = image.astype(np.uint8)
    return image

In [16]:
# Fungsi untuk klip dan simpan citra per desa
def clip_and_save(desa, src, output_folder):
    desa_name = desa['iddesa']
    geom = desa['geometry']
    
    # Klip citra satelit menggunakan geometri desa
    out_image, out_transform = mask(src, [geom], crop=True)
    
    # Normalisasi dan konversi tipe data
    out_image = normalize_and_convert(out_image)
    
    # Buat citra dengan saluran alpha (transparansi)
    alpha_channel = np.where(np.all(out_image == 0, axis=0), 0, 255).astype(np.uint8)
    out_image = np.concatenate([out_image, alpha_channel[np.newaxis, ...]], axis=0)
    
    # Simpan citra hasil klip ke format PNG dengan transparansi
    out_meta = src.meta.copy()
    out_meta.update({"driver": "PNG",
                     "dtype": "uint8",
                     "count": out_image.shape[0],
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})
    
    output_path = os.path.join(output_folder, f"{desa_name}.png")
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(out_image)
    
    # Plot dan simpan sebagai gambar tanpa sumbu dan latar belakang
    fig, ax = plt.subplots()
    ax.imshow(out_image.transpose(1, 2, 0))
    ax.axis('off')  
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0, transparent=True)
    plt.close(fig)

#### Dataset Landsat

In [17]:
# Muat citra satelit
tiff_file = "Landsat8-Daylight.tif"
src = rasterio.open(tiff_file)

# Folder output
output_folder = "OutputDay_Folder/Landsat8"
os.makedirs(output_folder, exist_ok=True)

# Iterasi melalui setiap desa dan simpan citra yang telah diklip
for index, desa in desa_gdf.iterrows():
    clip_and_save(desa, src, output_folder)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18152\2958332303.py:3: RuntimeWarning: invalid value encountered in divide
  image = np.array([(band - band.min()) / (band.max() - band.min()) * 255 for band in image])
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18152\2958332303.py:5: RuntimeWarning: invalid value encountered in cast
  image = image.astype(np.uint8)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18152\2958332303.py:3: RuntimeWarning: invalid value encountered in divide
  image = np.array([(band - band.min()) / (band.max() - band.min()) * 255 for band in image])
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18152\2958332303.py:5: RuntimeWarning: invalid value encountered in cast
  image = image.astype(np.uint8)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18152\2958332303.py:3: RuntimeWarning: invalid value encountered in divide
  image = np.array([(band - band.min()) / (band.max() - band.min()) * 255 for band in image])
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18152\29583

#### Dataset Sentinel-2

In [18]:
tiff_file = "Sentinel2-Daylight.tif"
src = rasterio.open(tiff_file)

# Folder output
output_folder = "OutputDay_Folder/Sentinel2"
os.makedirs(output_folder, exist_ok=True)

# Iterasi melalui setiap desa dan simpan citra yang telah diklip
for index, desa in desa_gdf.iterrows():
    clip_and_save(desa, src, output_folder)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18152\2958332303.py:3: RuntimeWarning: invalid value encountered in divide
  image = np.array([(band - band.min()) / (band.max() - band.min()) * 255 for band in image])
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18152\2958332303.py:5: RuntimeWarning: invalid value encountered in cast
  image = image.astype(np.uint8)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18152\2958332303.py:3: RuntimeWarning: invalid value encountered in divide
  image = np.array([(band - band.min()) / (band.max() - band.min()) * 255 for band in image])
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18152\2958332303.py:5: RuntimeWarning: invalid value encountered in cast
  image = image.astype(np.uint8)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18152\2958332303.py:3: RuntimeWarning: invalid value encountered in divide
  image = np.array([(band - band.min()) / (band.max() - band.min()) * 255 for band in image])
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18152\29583

### Pelabelan Data 

In [3]:
# Baca data dari file CSV
csv_file = 'idm_baru - Sheet1.csv' 
df = pd.read_csv(csv_file)

# Tampilkan atribut (kolom-kolom) dari CSV
print("Atribut (Kolom) yang ada di file CSV:")
print(df.columns)

Atribut (Kolom) yang ada di file CSV:
Index(['KABUPATEN', 'KECAMATAN', 'KODE BPS', 'DESA', 'IKS', 'IKE', 'IKL',
       'IDM', 'KATEGORI IDM', 'BINARY STATUS', 'matches'],
      dtype='object')


In [20]:
# Direktori sumber di mana file-file asli berada
source_folder_landsat = 'OutputDay_Folder/Landsat8'
source_folder_sentinel = 'OutputDay_Folder/Sentinel2'

# Direktori tujuan di mana file-file akan dipindahkan
destination_base_folder = 'Dataset/Daylight'

# Buat folder tujuan jika belum ada
os.makedirs(destination_base_folder, exist_ok=True)

# Dapatkan daftar label unik
labels = df['BINARY STATUS'].unique()

# Buat folder berdasarkan label jika belum ada
for label in labels:
    label_folder = os.path.join(destination_base_folder, label)
    os.makedirs(label_folder, exist_ok=True)

# Fungsi untuk memindahkan file
def move_files(row, source_folder, suffix, original_suffix):
    original_file_name = str(row['KODE BPS']) + original_suffix
    destination_file_name = str(row['KODE BPS']) + suffix
    label = row['BINARY STATUS']
    
    # Buat path sumber dan tujuan
    source_path = os.path.join(source_folder, original_file_name)
    label_folder = os.path.join(destination_base_folder, label)
    destination_path = os.path.join(label_folder, destination_file_name)
    
    # Pindahkan file ke folder tujuan
    if os.path.exists(source_path):
        shutil.move(source_path, destination_path)
    #     print(f"Memindahkan {original_file_name} ke {destination_path}")
    # else:
    #     print(f"File {original_file_name} tidak ditemukan di {source_folder}")

# Iterasi melalui setiap baris di DataFrame
for index, row in df.iterrows():
    move_files(row, source_folder_landsat, '-l.png', '.png')
    move_files(row, source_folder_sentinel, '-s.png', '.png')

print("Proses pelabelan selesai.")

Proses pelabelan selesai.


### Klip Citra Nightlight dari Dataset VIIRS Sesuai dengan Batas Desa (SHP)

In [17]:
# Fungsi untuk konversi tipe data tanpa normalisasi
def convert_to_uint8(image):
    if image.dtype != np.uint8:
        image = image.astype(np.uint8)
    return image

In [18]:
# Fungsi untuk klip dan simpan sebagai PNG
def clip_and_save(desa, src, output_folder):
    desa_name = desa['iddesa']
    geom = desa['geometry']
    
    # Klip citra satelit menggunakan geometri desa
    out_image, out_transform = mask(src, [mapping(geom)], crop=True)
    
    # Konversi tipe data tanpa normalisasi
    out_image = convert_to_uint8(out_image)
    
    # Buat citra dengan saluran alpha (transparansi)
    alpha_channel = np.where(out_image[0] == 0, 0, 255).astype(np.uint8)
    out_image = np.concatenate([out_image, alpha_channel[np.newaxis, ...]], axis=0)
    
    # Simpan citra hasil klip ke format PNG dengan transparansi
    out_meta = src.meta.copy()
    out_meta.update({
        "driver": "PNG",
        "dtype": "uint8",
        "count": out_image.shape[0],
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform
    })
    
    output_path = os.path.join(output_folder, f"{desa_name}.png")
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(out_image)
    
    # Plot dan simpan sebagai gambar tanpa sumbu dan latar belakang
    fig, ax = plt.subplots()
    ax.imshow(out_image[0], cmap='gray', vmin=0, vmax=255)
    ax.axis('off')  # Menghapus sumbu
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0, transparent=True)
    plt.close(fig)



In [20]:
# Muat citra satelit
tiff_file = "VIIRS-NightLight.tif"
src = rasterio.open(tiff_file)

# Folder output
output_folder = "OutputNight_Folder"
os.makedirs(output_folder, exist_ok=True)

# Iterasi melalui setiap desa dan simpan citra yang telah diklip
for index, desa in desa_gdf.iterrows():
    clip_and_save(desa, src, output_folder)

# Tutup file TIFF
src.close()

#### Hapus pixel black dari citra malam

In [21]:
def remove_black_background(image_path):
    image = Image.open(image_path).convert("RGBA")
    data = np.array(image)
    
    # Replace black with transparency
    red, green, blue, alpha = data.T
    black_areas = (red == 0) & (green == 0) & (blue == 0)
    data[..., :-1][black_areas.T] = (0, 0, 0)
    data[..., -1][black_areas.T] = 0
    
    # Save the image with a transparent background
    new_image = Image.fromarray(data)
    new_image.save(image_path)

In [22]:
def process_folder(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".png"):
            file_path = os.path.join(folder_path, file_name)
            remove_black_background(file_path)
            print(f"Processed {file_path}")

process_folder(output_folder)

Processed OutputNight_Folder\1111010001.png
Processed OutputNight_Folder\1111010002.png
Processed OutputNight_Folder\1111010003.png
Processed OutputNight_Folder\1111010004.png
Processed OutputNight_Folder\1111010005.png
Processed OutputNight_Folder\1111010006.png
Processed OutputNight_Folder\1111010007.png
Processed OutputNight_Folder\1111010008.png
Processed OutputNight_Folder\1111010009.png
Processed OutputNight_Folder\1111010010.png
Processed OutputNight_Folder\1111010011.png
Processed OutputNight_Folder\1111010012.png
Processed OutputNight_Folder\1111010013.png
Processed OutputNight_Folder\1111010014.png
Processed OutputNight_Folder\1111010015.png
Processed OutputNight_Folder\1111010016.png
Processed OutputNight_Folder\1111010017.png
Processed OutputNight_Folder\1111010018.png
Processed OutputNight_Folder\1111010019.png
Processed OutputNight_Folder\1111010020.png
Processed OutputNight_Folder\1111010021.png
Processed OutputNight_Folder\1111010022.png
Processed OutputNight_Folder\111

In [7]:
output_folder = "OutputNight_Folder"

# Direktori tujuan di mana file-file akan dipindahkan
destination_base_folder = 'Dataset/NTL'

# Iterasi melalui setiap baris di DataFrame
for index, row in df.iterrows():
    move_files(row, output_folder, '.png', '.png')

print("Proses pelabelan selesai.")

Proses pelabelan selesai.
